In [2]:
import pandas as pd
from clickhouse_connect import get_client

In [6]:
def create_date_dimension(start_date, end_date):
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    df = pd.DataFrame(dates, columns=['Date'])

    df['Year'] = df['Date'].dt.year
    df['Quarter'] = df['Date'].dt.quarter
    df['Month'] = df['Date'].dt.month
    df['DayOfMonth'] = df['Date'].dt.day
    df['DayOfWeek'] = df['Date'].dt.dayofweek + 1  # Понедельник = 1, Воскресенье = 7
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 6 else 0)
    
    # Пример праздников
    holidays = ['2023-12-25', '2023-01-01', '2023-07-04']
    df['IsHoliday'] = df['Date'].dt.strftime('%Y-%m-%d').isin(holidays).astype(int)

    return df

In [7]:
date_dimension = create_date_dimension('2023-01-01', '2029-12-31')

In [8]:
date_dimension

,Date,Year,Quarter,Month,DayOfMonth,DayOfWeek,IsWeekend,IsHoliday
0,2023-01-01,2023,1,1,1,7,1,1
1,2023-01-02,2023,1,1,2,1,0,0
2,2023-01-03,2023,1,1,3,2,0,0
3,2023-01-04,2023,1,1,4,3,0,0
4,2023-01-05,2023,1,1,5,4,0,0
...,...,...,...,...,...,...,...,...
2552,2029-12-27,2029,4,12,27,4,0,0
2553,2029-12-28,2029,4,12,28,5,0,0
2554,2029-12-29,2029,4,12,29,6,1,0
2555,2029-12-30,2029,4,12,30,7,1,0


In [13]:
client = get_client(
    host='10.1.11.65',
    username='default',  
    password='pegas_warehouse2025',  
    database='loaders_olap' 
)

In [16]:
client.command('''
    CREATE TABLE IF NOT EXISTS loaders_olap.calendar (
        Date Date,
        Year Int32,
        Month Int32,
        DayOfMonth Int32,
        DayOfWeek Int32,
        WeekOfYear Int32,
        IsWeekend Int32,
        IsHoliday Int32
    ) ENGINE = MergeTree() ORDER BY Date;
''')

In [20]:
client.insert_df('calendar', date_dimension)